In [1]:
import pandas as pd

In [25]:
movies = pd.read_csv('raw_data/movies.csv')
actors = pd.read_csv('raw_data/actors.csv')
crew = pd.read_csv('raw_data/crew.csv')
languages = pd.read_csv('raw_data/languages.csv')
genres = pd.read_csv('raw_data/genres.csv')
studios = pd.read_csv('raw_data/studios.csv')
# themes = pd.read_csv('raw_data/themes.csv')
# releases = pd.read_csv('raw_data/releases.csv')
countries = pd.read_csv('raw_data/countries.csv')

In [26]:
movies = movies.drop(columns='tagline', axis=1)

In [27]:
actors = actors.drop(columns='role', axis=1)
actors = actors.dropna()

In [18]:
# crew.isnull().sum()
# crew.role.unique()

id      0
role    0
name    1
dtype: int64

In [28]:
# movies.head()
# id = 1000001
# temp = crew[crew['id']==1000001]
crew = crew[crew['role'].isin(['Director', 'Producer', 'Writer', 'Cinematography', 'Composer'])] #'Songs'
#consider relabel those with small proportion to new category "Others"
#embeddings make sense, but we will have little control
#TODO: we check later how we deal with this feature!

In [41]:
languages = languages[languages['type'].isin(['Language', 'Primary language'])].drop(columns='type')

In [7]:
# genres.genre.unique() #OneHotEncoder

In [ ]:
# studios.studio.value_counts()/studios.studio.value_counts().sum()
#consider relabel those with small proportion to new category "Others" or drop them
#embeddings make sense, but we will have little control
#TODO: we check later how we deal with this feature!

studio
BBC                      0.005241
Warner Bros. Pictures    0.004743
Columbia Pictures        0.004416
ARTE                     0.004157
Metro-Goldwyn-Mayer      0.004106
                           ...   
Island Creek Pictures    0.000001
Extremo Sur Films        0.000001
KHF Media                0.000001
Kude Media               0.000001
Cineploit Pictures       0.000001
Name: count, Length: 160881, dtype: float64

In [ ]:
# countries # categorical, high-cardinality, embeddings
# countries # if we chose studios, this one can be left out due to collinearity

,id,country
0,1000001,UK
1,1000001,USA
2,1000002,South Korea
3,1000003,USA
4,1000004,Germany
...,...,...
693471,1941593,China
693472,1941594,USA
693473,1941595,USA
693474,1941596,China


In [30]:
new_crew = (
    crew.groupby('id')
    .apply(lambda x: x.groupby('role')['name'].apply(list).to_dict())
    .reset_index(name='crew_dict')
)

/tmp/ipykernel_52055/2434692008.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.groupby('role')['name'].apply(list).to_dict())


In [33]:
new_genres = (
    genres.groupby('id')['genre']
    .apply(list)  # Aggregates genres into a list
    .reset_index(name='genre_list')  # Converts to DataFrame and renames the column
)

In [34]:
new_studios = (
    studios.groupby('id')['studio']
    .apply(list)  # Aggregates genres into a list
    .reset_index(name='studio_list')  # Converts to DataFrame and renames the column
)

In [35]:
new_actors = (
    actors.groupby('id')['name']
    .apply(list)  # Aggregates genres into a list
    .reset_index(name='actor_list')  # Converts to DataFrame and renames the column
)

In [43]:
data = movies.merge(new_genres, how='left', on='id').merge(new_actors, how='left', on='id').merge(languages, how='left', on='id').merge(new_studios, how='left', on='id').merge(new_crew, how='left', on='id')

In [47]:
data.to_csv('compiled_movies.csv', index=False)

In [ ]:
data.

In [ ]:
# Incremental Recall
# python surprise
# Qualitative not

- dropna, fillna: imputer
- solve outliners (year range): drop those that have 
- 

In [56]:
data.date.isnull().count()

np.int64(941597)

In [57]:
# data.date.sort_values(ascending=False)
data.date.sort_values()

12087     1874.0
59978     1878.0
87422     1878.0
88060     1878.0
51499     1878.0
           ...  
941592       NaN
941593       NaN
941594       NaN
941595       NaN
941596       NaN
Name: date, Length: 941597, dtype: float64

In [ ]:
genres['genre_list'] = genres.groupby('id')['genre'].apply(lambda x: ', '.join(x)).reset_index()
# movies_with_genres = movies.merge(aggregated_genres, on='id', how='left')
# Convert the genre strings into lists
# genres['genre_list'] = genres['genre'].str.split(', ')
# Handle missing values
movies_with_genres['genre_list'] = movies_with_genres['genre_list'].apply(lambda x: x if isinstance(x, list) else [])
movies_with_genres.drop(columns="genre", axis=1, inplace=True)
unique_genres = set([genre for sublist in movies_with_genres['genre_list'] for genre in sublist])
unique_genres = sorted(list(unique_genres))
print(unique_genres)
for genre in unique_genres:
    movies_with_genres[genre] = movies_with_genres['genre_list'].apply(lambda x: 1 if genre in x else 0)

In [ ]:
data = movies.merge(actors, how='left', on='id').merge()

In [60]:
B_movie = pd.read_csv('raw_data/Set B/movie.csv')

In [61]:
B_rating = pd.read_csv('raw_data/Set B/rating.csv')

In [62]:
B_movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [63]:
B_data = B_movie.merge(B_rating, how='left', on='movieId')

In [ ]:
import re

In [67]:
B_data['year'] = B_data['title'].str.extract(r'\((\d{4})\)')

In [69]:
B_data['new_title'] = B_data['title'].str.replace(r'\s*\(\d{4}\)', '', regex=True)

In [70]:
B_data

,movieId,title,genres,userId,rating,timestamp,year,new_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47,1995,Toy Story
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52,1995,Toy Story
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51,1995,Toy Story
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47,1995,Toy Story
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41,1995,Toy Story
...,...,...,...,...,...,...,...,...
20000792,131254,Kein Bund für's Leben (2007),Comedy,79570.0,4.0,2015-03-30 19:32:59,2007,Kein Bund für's Leben
20000793,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,79570.0,4.0,2015-03-30 19:48:08,2002,"Feuer, Eis & Dosenbier"
20000794,131258,The Pirates (2014),Adventure,28906.0,2.5,2015-03-30 19:56:32,2014,The Pirates
20000795,131260,Rentun Ruusu (2001),(no genres listed),65409.0,3.0,2015-03-30 19:57:46,2001,Rentun Ruusu


In [89]:
# new_B = B_data[['new_title']].drop_duplicates()
new_B = B_data[['title']].drop_duplicates()

In [91]:
new_B.title.nunique()

27262

In [80]:
data.name.nunique()

795421

In [ ]:
new_A = data['date'].dropna()

In [92]:
data['key_b'] = data['name'] + ' ' + str(int(data['date']))

,id,name,date,description,minute,rating,genre_list,actor_list,language,studio_list,crew_dict
0,1000001,Barbie,2023.0,Barbie and Ken are having the time of their li...,114.0,3.86,"[Comedy, Adventure]","[Margot Robbie, Ryan Gosling, America Ferrera,...",English,"[LuckyChap Entertainment, Heyday Films, NB/GG ...","{'Cinematography': ['Rodrigo Prieto'], 'Compos..."
1,1000002,Parasite,2019.0,"All unemployed, Ki-taek's family takes peculia...",133.0,4.56,"[Comedy, Thriller, Drama]","[Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong, Ch...",Korean,[Barunson E&A],"{'Cinematography': ['Hong Kyung-pyo'], 'Compos..."
2,1000003,Everything Everywhere All at Once,2022.0,An aging Chinese immigrant is swept up in an i...,140.0,4.30,"[Science Fiction, Adventure, Comedy, Action]","[Michelle Yeoh, Ke Huy Quan, Stephanie Hsu, Ja...",English,"[IAC Films, AGBO, Ley Line Entertainment, Year...","{'Cinematography': ['Larkin Seiple'], 'Compose..."
3,1000004,Fight Club,1999.0,A ticking-time-bomb insomniac and a slippery s...,139.0,4.27,[Drama],"[Edward Norton, Brad Pitt, Helena Bonham Carte...",English,"[Fox 2000 Pictures, Regency Enterprises, The L...","{'Cinematography': ['Jeff Cronenweth'], 'Compo..."
4,1000005,La La Land,2016.0,"Mia, an aspiring actress, serves lattes to mov...",129.0,4.09,"[Drama, Comedy, Music, Romance]","[Ryan Gosling, Emma Stone, John Legend, Rosema...",English,"[Summit Entertainment, Black Label Media, Gilb...","{'Cinematography': ['Linus Sandgren'], 'Compos..."
...,...,...,...,...,...,...,...,...,...,...,...
941592,1941593,神笛,NaN,NaN,NaN,NaN,NaN,NaN,Chinese,NaN,NaN
941593,1941594,蟲極道蜜団子抗争編 壱ノ巻,NaN,Shinjuku forest at night. In the sap taverns o...,30.0,NaN,NaN,NaN,English,NaN,NaN
941594,1941595,蟲極道蜜団子抗争編 弐ノ巻,NaN,"The city that never sleeps, where insects gath...",30.0,NaN,NaN,NaN,English,NaN,NaN
941595,1941596,重生,NaN,"In a world where order has broken down, darkne...",NaN,NaN,"[Action, Crime]","[Nick Cheung, Ethan Juan, Sandrine Pinna, Marc...",Chinese,"[上海猫眼影业有限公司, 坏小子（北京）传媒有限公司, 亚太国影（重庆）文化传媒有限公司, ...","{'Cinematography': ['Kenny Tse'], 'Composer': ..."


In [88]:
data.groupby('name').size().reset_index(name='frequency').sort_values(by='frequency', ascending=False)

,name,frequency
277543,Home,143
722444,Untitled,118
44511,Alone,108
421616,Mother,98
380233,Love,76
...,...,...
275232,History of Manawan - Part Two,1
275233,History of Manawan – Part Two,1
275234,History of Memory,1
275235,History of Micronesia,1


np.int64(10)

In [75]:
data.merge(new_B, how='inner',)

,id,name,date,description,minute,rating,genre_list,actor_list,language,studio_list,crew_dict
0,1000001,Barbie,2023.0,Barbie and Ken are having the time of their li...,114.0,3.86,"[Comedy, Adventure]","[Margot Robbie, Ryan Gosling, America Ferrera,...",English,"[LuckyChap Entertainment, Heyday Films, NB/GG ...","{'Cinematography': ['Rodrigo Prieto'], 'Compos..."
1,1000002,Parasite,2019.0,"All unemployed, Ki-taek's family takes peculia...",133.0,4.56,"[Comedy, Thriller, Drama]","[Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong, Ch...",Korean,[Barunson E&A],"{'Cinematography': ['Hong Kyung-pyo'], 'Compos..."
2,1000003,Everything Everywhere All at Once,2022.0,An aging Chinese immigrant is swept up in an i...,140.0,4.30,"[Science Fiction, Adventure, Comedy, Action]","[Michelle Yeoh, Ke Huy Quan, Stephanie Hsu, Ja...",English,"[IAC Films, AGBO, Ley Line Entertainment, Year...","{'Cinematography': ['Larkin Seiple'], 'Compose..."
3,1000004,Fight Club,1999.0,A ticking-time-bomb insomniac and a slippery s...,139.0,4.27,[Drama],"[Edward Norton, Brad Pitt, Helena Bonham Carte...",English,"[Fox 2000 Pictures, Regency Enterprises, The L...","{'Cinematography': ['Jeff Cronenweth'], 'Compo..."
4,1000005,La La Land,2016.0,"Mia, an aspiring actress, serves lattes to mov...",129.0,4.09,"[Drama, Comedy, Music, Romance]","[Ryan Gosling, Emma Stone, John Legend, Rosema...",English,"[Summit Entertainment, Black Label Media, Gilb...","{'Cinematography': ['Linus Sandgren'], 'Compos..."
...,...,...,...,...,...,...,...,...,...,...,...
941592,1941593,神笛,NaN,NaN,NaN,NaN,NaN,NaN,Chinese,NaN,NaN
941593,1941594,蟲極道蜜団子抗争編 壱ノ巻,NaN,Shinjuku forest at night. In the sap taverns o...,30.0,NaN,NaN,NaN,English,NaN,NaN
941594,1941595,蟲極道蜜団子抗争編 弐ノ巻,NaN,"The city that never sleeps, where insects gath...",30.0,NaN,NaN,NaN,English,NaN,NaN
941595,1941596,重生,NaN,"In a world where order has broken down, darkne...",NaN,NaN,"[Action, Crime]","[Nick Cheung, Ethan Juan, Sandrine Pinna, Marc...",Chinese,"[上海猫眼影业有限公司, 坏小子（北京）传媒有限公司, 亚太国影（重庆）文化传媒有限公司, ...","{'Cinematography': ['Kenny Tse'], 'Composer': ..."


In [76]:
data.merge(new_B, how='inner', left_on='name', right_on='new_title')

,id,name,date,description,minute,rating,genre_list,actor_list,language,studio_list,crew_dict,new_title
0,1000002,Parasite,2019.0,"All unemployed, Ki-taek's family takes peculia...",133.0,4.56,"[Comedy, Thriller, Drama]","[Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong, Ch...",Korean,[Barunson E&A],"{'Cinematography': ['Hong Kyung-pyo'], 'Compos...",Parasite
1,1000004,Fight Club,1999.0,A ticking-time-bomb insomniac and a slippery s...,139.0,4.27,[Drama],"[Edward Norton, Brad Pitt, Helena Bonham Carte...",English,"[Fox 2000 Pictures, Regency Enterprises, The L...","{'Cinematography': ['Jeff Cronenweth'], 'Compo...",Fight Club
2,1000007,Interstellar,2014.0,The adventures of a group of explorers who mak...,169.0,4.35,"[Science Fiction, Drama, Adventure]","[Matthew McConaughey, Anne Hathaway, Michael C...",English,"[Legendary Pictures, Syncopy, Lynda Obst Produ...","{'Cinematography': ['Hoyte van Hoytema'], 'Com...",Interstellar
3,1000008,Joker,2019.0,"During the 1980s, a failed stand-up comedian i...",122.0,3.85,"[Crime, Drama, Thriller]","[Joaquin Phoenix, Robert De Niro, Zazie Beetz,...",English,"[Warner Bros. Pictures, Joint Effort, Village ...","{'Cinematography': ['Lawrence Sher'], 'Compose...",Joker
4,1000009,Dune,2021.0,"Paul Atreides, a brilliant and gifted young ma...",155.0,3.90,"[Science Fiction, Adventure]","[Timothée Chalamet, Rebecca Ferguson, Oscar Is...",English,[Legendary Pictures],"{'Cinematography': ['Greig Fraser'], 'Composer...",Dune
...,...,...,...,...,...,...,...,...,...,...,...,...
46175,1941173,Ghost,NaN,A Group of some people hijacked the jail in th...,132.0,NaN,NaN,NaN,English,NaN,NaN,Ghost
46176,1941307,Memento,NaN,Dedicated to the living and the dead. Video fi...,64.0,NaN,NaN,"[Vladimír Hajdu, Laura Kovalová, Jozef Mokoš, ...",Slovak,[Československá televízia Bratislava],{'Director': ['Samuel Ivaška']},Memento
46177,1941319,The Missing,NaN,The death of Eric’s uncle triggers a suppresse...,NaN,NaN,NaN,NaN,English,NaN,NaN,The Missing
46178,1941396,Rebound,NaN,"Theo, a college student torn between his past ...",NaN,NaN,"[Romance, Drama]","[Joshua Chan, Maximos Sean Jalali, Darian Madhi]",English,[Tarot 77 Entertainment],"{'Cinematography': ['Henry Shriber'], 'Compose...",Rebound


# SUMMARY
- import csv files
- 

ADVISE: high-cardiality --> may make sense to drop the rare guys or not